In [47]:
import requests
import json
import time
import pandas as pd

class FreeTierLimitReached(Exception):
    pass

def get_coffee_shops(api_key, location, radius=1500, keyword="coffee"):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "key": api_key,
        "location": location,
        "radius": radius,
        "keyword": keyword
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, response.text)
        return None

def get_place_details(api_key, place_id):
    url = "https://maps.googleapis.com/maps/api/place/details/json"
    params = {
        "key": api_key,
        "place_id": place_id,
        "fields": "name,geometry,opening_hours,rating,url"
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error fetching place details:", response.status_code, response.text)
        return None

def extract_place_details(places_data, api_key):
    place_details = []
    if places_data and 'results' in places_data:
        for place in places_data["results"]:
            place_id = place["place_id"]
            details = get_place_details(api_key, place_id)
            if details and 'result' in details:
                result = details['result']
                name = result.get("name")
                lat = result["geometry"]["location"]["lat"]
                lng = result["geometry"]["location"]["lng"]
                rating = result.get("rating", "No rating available")
                maps_url = result.get("url")
                opening_hours = result.get("opening_hours", {}).get("weekday_text", "No hours available")
                place_details.append((name, lat, lng, rating, maps_url, opening_hours))
    return place_details

def get_next_page(api_key, next_page_token):
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "key": api_key,
        "pagetoken": next_page_token
    }
    time.sleep(2)  # Add delay to ensure next_page_token is valid
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code, response.text)
        return None

def check_free_tier_limit(request_count, limit=1000):
    if request_count >= limit:
        raise FreeTierLimitReached("Free tier limit reached. Stopping script to avoid charges.")

if __name__ == "__main__":
    api_key = "my_secret_api_key"
    location = "27.7676,-82.6403"  # Example coordinates for St. Petersburg, FL
    radius = 5000  # Search radius in meters
    request_count = 0
    free_tier_limit = 1000  # Adjust based on your free tier limit
    place_details = []

    try:
        places_data = get_coffee_shops(api_key, location, radius)
        request_count += 1
        check_free_tier_limit(request_count, free_tier_limit)
        print("Initial API response:", json.dumps(places_data, indent=2))

        place_details.extend(extract_place_details(places_data, api_key))

        # Handle pagination if more results are available
        while places_data and 'next_page_token' in places_data:
            next_page_token = places_data['next_page_token']
            print("Fetching next page of results with token:", next_page_token)
            places_data = get_next_page(api_key, next_page_token)
            request_count += 1
            check_free_tier_limit(request_count, free_tier_limit)
            print("Next page API response:", json.dumps(places_data, indent=2))
            place_details.extend(extract_place_details(places_data, api_key))

        # Save the results to a DataFrame and CSV file
        df = pd.DataFrame(place_details, columns=["Name", "Latitude", "Longitude", "Rating", "Google Maps URL", "Opening Hours"])

        # Print the DataFrame to verify
        print(df)

    except FreeTierLimitReached as e:
        print(e)


Initial API response: {
  "html_attributions": [],
  "next_page_token": "AelY_CtuRaF5e_NqBL9iTtZi_5suI9R-sr_tuio4caLpqyTr1fihNASHOVH_-QGeV80d71dPFM9qmoHi6guTZ7XvmWv8563GNrF7-ePNhmk8r0AnGFePTIvd5Ms7zeKaIWxMBiy4SO8mnVSQWojYnlokaZCqytAaoaKi6XIM5wMjz_hvTTASHh3Cm3oLaBV0XF4a4EEru0gZJlK5dvUC2Kxksbj3ClAdEfBt4tlxHQm8wgJdy4NPhRwZwaCtrE-PcHYrq6hHIl3ZOTIljmkKoMW7PzTUEssIq451cOho3qH-D4goGKIuO1BAYURXg3wEainSQZtzlD87Xkv3p-_k5YEW1uab32R7cBIrbor-7d58cJMX18fcBoJQDTJNH6geCzM52BXmhlboLNX6yYBkIt51dM8SYXn9go6M4Bdi624NWluhIkT3ezYk1-g",
  "results": [
    {
      "business_status": "OPERATIONAL",
      "geometry": {
        "location": {
          "lat": 27.7736765,
          "lng": -82.6335508
        },
        "viewport": {
          "northeast": {
            "lat": 27.77497597989272,
            "lng": -82.63220157010728
          },
          "southwest": {
            "lat": 27.77227632010728,
            "lng": -82.63490122989272
          }
        }
      },
      "icon": "https://maps.gstatic.com/m

In [56]:
df

,Name,Latitude,Longitude,Rating,Google Maps URL,Opening Hours
0,Paradeco Coffee Roasters,27.773676,-82.633551,4.7,https://maps.google.com/?cid=361888425776248428,"Monday: 7:00 AM – 5:00 PM, Tuesday: 7:00 AM – ..."
1,Caribou Coffee,27.771378,-82.644084,4.7,https://maps.google.com/?cid=7287627397810253140,"Monday: 7:00 AM – 5:30 PM, Tuesday: 7:00 AM – ..."
2,The Blend (34th St N) Coffee & Cocktails,27.796435,-82.679974,4.4,https://maps.google.com/?cid=15427381742529607274,"Monday: 6:30 AM – 7:00 PM, Tuesday: 6:30 AM – ..."
3,2D Cafe,27.771237,-82.661880,4.5,https://maps.google.com/?cid=11652252313769637407,"Monday: Closed, Tuesday: Closed, Wednesday: 8:..."
4,Moon Mountain,27.771904,-82.635862,4.9,https://maps.google.com/?cid=13374119603334673592,"Monday: 6:30 AM – 11:00 PM, Tuesday: 6:30 AM –..."
5,Starbucks,27.808264,-82.638232,4,https://maps.google.com/?cid=16304193782206337378,"Monday: 5:00 AM – 8:00 PM, Tuesday: 5:00 AM – ..."
6,Starbucks,27.726000,-82.680446,4.1,https://maps.google.com/?cid=15322416243382616832,"Monday: 5:00 AM – 8:00 PM, Tuesday: 5:00 AM – ..."
7,Foxtail Coffee Co.,27.779776,-82.638733,4.3,https://maps.google.com/?cid=13987541391888552573,"Monday: 6:30 AM – 9:00 PM, Tuesday: 6:30 AM – ..."
8,Raining Berries St. Pete,27.771237,-82.655538,4.3,https://maps.google.com/?cid=15816009991780413297,"Monday: 7:00 AM – 8:00 PM, Tuesday: 7:00 AM – ..."
9,The Blend (Old Northeast) Coffee & Cocktails,27.784199,-82.638216,4.4,https://maps.google.com/?cid=12808946830342218904,"Monday: 7:00 AM – 7:00 PM, Tuesday: 7:00 AM – ..."


In [58]:
# Clean the Opening Hours column
df["Opening Hours"] = df["Opening Hours"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)
df

,Name,Latitude,Longitude,Rating,Google Maps URL,Opening Hours
0,Paradeco Coffee Roasters,27.773676,-82.633551,4.7,https://maps.google.com/?cid=361888425776248428,"Monday: 7:00 AM – 5:00 PM, Tuesday: 7:00 AM – ..."
1,Caribou Coffee,27.771378,-82.644084,4.7,https://maps.google.com/?cid=7287627397810253140,"Monday: 7:00 AM – 5:30 PM, Tuesday: 7:00 AM – ..."
2,The Blend (34th St N) Coffee & Cocktails,27.796435,-82.679974,4.4,https://maps.google.com/?cid=15427381742529607274,"Monday: 6:30 AM – 7:00 PM, Tuesday: 6:30 AM – ..."
3,2D Cafe,27.771237,-82.661880,4.5,https://maps.google.com/?cid=11652252313769637407,"Monday: Closed, Tuesday: Closed, Wednesday: 8:..."
4,Moon Mountain,27.771904,-82.635862,4.9,https://maps.google.com/?cid=13374119603334673592,"Monday: 6:30 AM – 11:00 PM, Tuesday: 6:30 AM –..."
5,Starbucks,27.808264,-82.638232,4,https://maps.google.com/?cid=16304193782206337378,"Monday: 5:00 AM – 8:00 PM, Tuesday: 5:00 AM – ..."
6,Starbucks,27.726000,-82.680446,4.1,https://maps.google.com/?cid=15322416243382616832,"Monday: 5:00 AM – 8:00 PM, Tuesday: 5:00 AM – ..."
7,Foxtail Coffee Co.,27.779776,-82.638733,4.3,https://maps.google.com/?cid=13987541391888552573,"Monday: 6:30 AM – 9:00 PM, Tuesday: 6:30 AM – ..."
8,Raining Berries St. Pete,27.771237,-82.655538,4.3,https://maps.google.com/?cid=15816009991780413297,"Monday: 7:00 AM – 8:00 PM, Tuesday: 7:00 AM – ..."
9,The Blend (Old Northeast) Coffee & Cocktails,27.784199,-82.638216,4.4,https://maps.google.com/?cid=12808946830342218904,"Monday: 7:00 AM – 7:00 PM, Tuesday: 7:00 AM – ..."


In [64]:
# add columns for mapbox parameters
columns_to_add = ['bearing', 'duration', 'zoom', 'pitch', 'speed', 'active']
for column in columns_to_add:
    df[column] = None

,Name,Latitude,Longitude,Rating,Google Maps URL,Opening Hours,bearing,duration,zoom,pitch,speed,active
0,Paradeco Coffee Roasters,27.773676,-82.633551,4.7,https://maps.google.com/?cid=361888425776248428,"Monday: 7:00 AM – 5:00 PM, Tuesday: 7:00 AM – ...",None,None,None,None,None,None
1,Caribou Coffee,27.771378,-82.644084,4.7,https://maps.google.com/?cid=7287627397810253140,"Monday: 7:00 AM – 5:30 PM, Tuesday: 7:00 AM – ...",None,None,None,None,None,None
2,The Blend (34th St N) Coffee & Cocktails,27.796435,-82.679974,4.4,https://maps.google.com/?cid=15427381742529607274,"Monday: 6:30 AM – 7:00 PM, Tuesday: 6:30 AM – ...",None,None,None,None,None,None
3,2D Cafe,27.771237,-82.661880,4.5,https://maps.google.com/?cid=11652252313769637407,"Monday: Closed, Tuesday: Closed, Wednesday: 8:...",None,None,None,None,None,None
4,Moon Mountain,27.771904,-82.635862,4.9,https://maps.google.com/?cid=13374119603334673592,"Monday: 6:30 AM – 11:00 PM, Tuesday: 6:30 AM –...",None,None,None,None,None,None
5,Starbucks,27.808264,-82.638232,4,https://maps.google.com/?cid=16304193782206337378,"Monday: 5:00 AM – 8:00 PM, Tuesday: 5:00 AM – ...",None,None,None,None,None,None
6,Starbucks,27.726000,-82.680446,4.1,https://maps.google.com/?cid=15322416243382616832,"Monday: 5:00 AM – 8:00 PM, Tuesday: 5:00 AM – ...",None,None,None,None,None,None
7,Foxtail Coffee Co.,27.779776,-82.638733,4.3,https://maps.google.com/?cid=13987541391888552573,"Monday: 6:30 AM – 9:00 PM, Tuesday: 6:30 AM – ...",None,None,None,None,None,None
8,Raining Berries St. Pete,27.771237,-82.655538,4.3,https://maps.google.com/?cid=15816009991780413297,"Monday: 7:00 AM – 8:00 PM, Tuesday: 7:00 AM – ...",None,None,None,None,None,None
9,The Blend (Old Northeast) Coffee & Cocktails,27.784199,-82.638216,4.4,https://maps.google.com/?cid=12808946830342218904,"Monday: 7:00 AM – 7:00 PM, Tuesday: 7:00 AM – ...",None,None,None,None,None,None


In [66]:
#df.to_csv("stpete_all_cafe_coords.csv",index=False)